In [3]:
import pandas as pd
import numpy as np
import os

In [237]:
usecols = ["Chr", "Start", "End", "Ref", "Alt", "Func.refGene", "Otherinfo10", "Otherinfo11"]
dtype = {"Chr":"category",
          "Start":np.int32,
          "End":np.int32,
          "Ref":"category",
          "Alt":"category",
          "Func.refGene":"category",
          "Otherinfo10":"category",
          "Otherinfo11":"string"}

In [589]:
chr_filename1 = "chr17_22241.csv"
chr_filename2 = "chr17_22334.csv"

df1_ = pd.read_csv(chr_filename1, usecols=usecols, dtype=dtype, sep='\t')
df1_ = pd.eval("DP = df1_.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=df1_)
df1_ = pd.eval("AF = df1_.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=df1_)
df2_ = pd.read_csv(chr_filename2, usecols=usecols, sep='\t')
df2_ = pd.eval("DP = df2_.Otherinfo11.str.split(';',3).str[2].str.split('DP=').str[1].astype('int32')", target=df2_)
df2_ = pd.eval("AF = df2_.Otherinfo11.str.split(';',5).str[4].str.split('AF=').str[1].astype('float')", target=df2_)

In [605]:
# this works on Chr, Ref, Alt, Func.refGene, Otherinfo10
# for Func.refGene, enter 'FuncrefGene' instead
def category_filters(df, export=False, inplace=False, **kwargs):
    for key, value in kwargs.items():
        if key == "FuncrefGene":
            df = df[df["Func.refGene"] == value]
        else:
            df = df[df[key] == value]
    if inplace:
        globals()['df1_'] = df
    if export:
        df.to_csv('cat_filter_dataframe.csv', index=False, header=True)
    return df

In [592]:
# for Func.refGene, enter 'FuncrefGene' instead 
display(category_filters(df1_,FuncrefGene="exonic", Otherinfo10="PASS"))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
106,chr17,410351,410351,G,T,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=133;VD=66;AF=0.496...,133,0.4962
107,chr17,410391,410391,T,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=117;VD=65;AF=0.555...,117,0.5556
108,chr17,410444,410444,C,A,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=152;VD=8;AF=0.0526...,152,0.0526
110,chr17,410508,410508,G,A,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=207;VD=98;AF=0.473...,207,0.4734
111,chr17,410632,410632,C,A,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=344;VD=5;AF=0.0145...,344,0.0145
...,...,...,...,...,...,...,...,...,...,...
50362,chr17,82831752,82831752,T,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=814;VD=388;AF=0.47...,814,0.4767
50449,chr17,82884148,82884148,A,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=382;VD=148;AF=0.38...,382,0.3874
50471,chr17,82905981,82905981,T,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=613;VD=311;AF=0.50...,613,0.5073
50472,chr17,82905989,82905989,G,T,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=671;VD=7;AF=0.0104...,671,0.0104


In [607]:
# this assumes greater than i.e 'DP = 15' means keep DP vals > 15
def num_filter(df, export=False, **kwargs):
    for key, value in kwargs.items():
        df.loc[:,key].where(df.loc[:,key] > value,inplace=True).dropna()
    if inplace:
        globals()['df1_'] = df
    if export:
        df.to_csv('cat_filter_dataframe.csv', index=False, header=True)
    return df

In [443]:
display(num_filter(df1_,DP=150,AF=.15))

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
0,chr17,137603,137603,C,T,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=521;VD=287;AF=0.55...,521.0,0.5509
1,chr17,137604,137604,G,A,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=506;VD=121;AF=0.23...,506.0,0.2391
3,chr17,138814,138814,-,C,intronic,Q10,SAMPLE=VE_22241;TYPE=Insertion;DP=191;VD=33;AF...,191.0,0.1728
4,chr17,138824,138824,C,A,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=186;VD=60;AF=0.322...,186.0,0.3226
8,chr17,156366,156366,A,G,exonic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=453;VD=216;AF=0.47...,453.0,0.4768
...,...,...,...,...,...,...,...,...,...,...
50517,chr17,82941513,82941513,G,T,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=453;VD=233;AF=0.51...,453.0,0.5143
50519,chr17,82957112,82957112,G,T,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=168;VD=86;AF=0.511...,168.0,0.5119
50593,chr17,83093088,83093088,G,-,intronic,Q10,SAMPLE=VE_22241;TYPE=Deletion;DP=377;VD=174;AF...,377.0,0.4615
50594,chr17,83093237,83093237,C,G,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=520;VD=519;AF=0.99...,520.0,0.9981


In [608]:
def subtraction(df1, df2, export=True):
    uqid_df1 = df1[df1.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()
    uqid_df2 = df2[df2.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()  
    uqid = pd.concat([uqid_df1, uqid_df2], ignore_index=True)
    uqid_df1_len, uqid_df2_len= len(uqid_df1.index), len(uqid_df2.index)
    idx = np.append(np.arange(0, uqid_df1_len), np.arange(0, uqid_df2_len))
    uqid_df = pd.DataFrame({'idx':idx}, index=uqid)
    uqid_df.reset_index(inplace=True)
    uqid_df = uqid_df.sort_index().groupby('index').filter(lambda x: len(x) == 1)
    idx = uqid_df['idx'].to_numpy()
    mid_val = idx[len(idx)//5:].argmin() + len(idx)//5
    inv_arr = lambda max_val,idx_arr: np.array(sorted(set(range(0, max_val)).difference(idx_arr)))
    df1=df1.drop(df1.index[inv_arr(uqid_df1_len, idx[:mid_val])])
    df2=df2.drop(df2.index[inv_arr(uqid_df2_len, idx[mid_val:])])
    if export:
        df1.to_csv('sub_df1.csv', index=False, header=True)
        df2.to_csv('sub_df2.csv', index=False, header=True)
    return [df1,df2]

In [604]:
sub_dfs = subtraction(df1_, df2_)
sub_df1 = sub_dfs[0]
sub_df2 = sub_dfs[1]

TypeError: 'NoneType' object is not subscriptable

In [588]:
display(sub_df1)
display(sub_df2)

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
2,chr17,138812,138812,-,G,intronic,Q10,SAMPLE=VE_22241;TYPE=Insertion;DP=194;VD=2;AF=...,194.0,NaN
7,chr17,156172,156172,C,A,intronic,p8;pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=146;VD=2;AF=0.0137...,NaN,NaN
9,chr17,159978,159978,-,T,intronic,PASS,SAMPLE=VE_22241;TYPE=Insertion;DP=2;VD=2;AF=1;...,NaN,1.0
11,chr17,164114,164114,G,A,intronic,Q10,SAMPLE=VE_22241;TYPE=SNV;DP=169;VD=3;AF=0.0178...,169.0,NaN
12,chr17,164127,164127,C,A,intronic,pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=184;VD=2;AF=0.0109...,184.0,NaN
...,...,...,...,...,...,...,...,...,...,...
50591,chr17,83091989,83091989,A,G,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=5;VD=5;AF=1;BIAS=0...,NaN,1.0
50592,chr17,83092970,83092970,C,T,intronic,pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=95;VD=3;AF=0.0316;...,NaN,NaN
50596,chr17,83093330,83093330,G,T,intronic,pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=126;VD=2;AF=0.0159...,NaN,NaN
50598,chr17,83094172,83094172,G,A,intronic,pSTD;Q10,SAMPLE=VE_22241;TYPE=SNV;DP=184;VD=2;AF=0.0109...,184.0,NaN


,Chr,Start,End,Ref,Alt,AF,DP,Func.refGene,Otherinfo10,Otherinfo11
2,chr17,137724,137724,G,A,NaN,NaN,NaN,NaN,NaN
3,chr17,138274,138274,G,T,NaN,NaN,NaN,NaN,NaN
4,chr17,138424,138424,C,A,NaN,NaN,NaN,NaN,NaN
5,chr17,138789,138789,G,T,NaN,NaN,NaN,NaN,NaN
8,chr17,143369,143369,G,A,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
58117,chr17,83092946,83092946,C,A,NaN,NaN,NaN,NaN,NaN
58121,chr17,83093400,83093400,G,T,NaN,NaN,NaN,NaN,NaN
58123,chr17,83094157,83094157,G,A,NaN,NaN,NaN,NaN,NaN
58124,chr17,83094669,83094669,G,T,NaN,NaN,NaN,NaN,NaN


In [579]:
x = (df1_.iloc[24385:24595])
y = (df2_.iloc[27128:27338])

uqid_df1 = x[x.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()
uqid_df2 = y[y.columns[1:5]].apply(lambda x: ''.join(x.astype(str)),axis=1).squeeze()  
uqid = pd.concat([uqid_df1, uqid_df2], ignore_index=True)

uqid_df1_len, uqid_df2_len= len(uqid_df1.index), len(uqid_df2.index)

idx = np.append(np.arange(0, uqid_df1_len), np.arange(0, uqid_df1_len))
uqid_df = pd.DataFrame({'idx':idx}, index=uqid)
print(idx)

uqid_df.reset_index(inplace=True)
uqid_df = uqid_df.sort_index().groupby('index').filter(lambda x: len(x) == 1)

idx = uqid_df['idx'].to_numpy()
print(idx)
mid_val = idx[len(idx)//5:].argmin() + len(idx)//5
inv_arr = lambda max_val,idx_arr: np.array(sorted(set(range(0, max_val)).difference(idx_arr)))
print(idx[mid_val])
print(idx[:mid_val])
print(inv_arr(uqid_df1_len, idx[:mid_val]))


x=x.drop(x.index[inv_arr(uqid_df1_len, idx[:mid_val])])
y=y.drop(y.index[inv_arr(uqid_df2_len, idx[mid_val:])])

display(x)
display(y)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209   0   1   2   3   4   5
   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23
  24  25  26  27  28  29  30  31  32  33  34  35  3

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
24390,chr17,41149549,41149563,GGATTCACAGCAATA,-,exonic,PASS,SAMPLE=VE_22241;TYPE=Deletion;DP=392;VD=4;AF=0...,392.0,NaN
24392,chr17,41160177,41160177,T,C,UTR3,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=594;VD=6;AF=0.0101...,594.0,NaN
24394,chr17,41167233,41167233,C,T,UTR3,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=4;VD=4;AF=1;BIAS=0...,NaN,1.0000
24395,chr17,41167649,41167649,C,A,exonic,p8,SAMPLE=VE_22241;TYPE=SNV;DP=386;VD=4;AF=0.0104...,386.0,NaN
24396,chr17,41167852,41167852,A,G,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=322;VD=5;AF=0.0155...,322.0,NaN
...,...,...,...,...,...,...,...,...,...,...
24590,chr17,41477356,41477356,A,G,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=48;VD=23;AF=0.4792...,NaN,0.4792
24591,chr17,41477644,41477644,T,C,exonic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=793;VD=10;AF=0.012...,793.0,NaN
24592,chr17,41477747,41477747,G,T,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=648;VD=9;AF=0.0139...,648.0,NaN
24593,chr17,41477808,41477808,C,G,intronic,PASS,SAMPLE=VE_22241;TYPE=SNV;DP=363;VD=4;AF=0.011;...,363.0,NaN


,Chr,Start,End,Ref,Alt,AF,DP,Func.refGene,Otherinfo10,Otherinfo11
27134,chr17,41160402,41160402,C,A,NaN,NaN,NaN,NaN,NaN
27135,chr17,41160427,41160427,G,T,NaN,NaN,NaN,NaN,NaN
27137,chr17,41167613,41167613,G,T,NaN,NaN,NaN,NaN,NaN
27138,chr17,41167777,41167777,G,T,NaN,NaN,NaN,NaN,NaN
27141,chr17,41167949,41167949,C,A,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
27325,chr17,41462677,41462677,C,A,NaN,NaN,NaN,NaN,NaN
27327,chr17,41464075,41464075,C,A,NaN,NaN,NaN,NaN,NaN
27332,chr17,41465485,41465485,G,T,NaN,NaN,NaN,NaN,NaN
27333,chr17,41465532,41465532,C,A,NaN,NaN,NaN,NaN,NaN


In [538]:
df1_drop_idxs = invert_arr(idx_arr[:sep_dfs], max_val)
df2_drop_idxs = invert_arr(idx_arr[sep_dfs:], max_val + 1) - len(uqid_df1.index)
print(df1_drop_idxs)
print(df2_drop_idxs)
x = x.drop(x.index[[df1_drop_idxs]])
y = y.drop(y.index[[df2_drop_idxs]])

display(x)
display(y)
# 4114950841149509CTTG
# 4114950841149509CTTG

In [496]:
sub_dfs = subtraction(df1_, df2_)
sub_df1 = sub_dfs[0]
sub_df2 = sub_dfs[1]
del sub_dfs

In [507]:
display(sub_df1.iloc[[24385]])
display(sub_df2.iloc[[27128]])
display(df1_.iloc[[24385]])

,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
24385,chr17,41149508,41149509,CT,TG,exonic,NM5.25,SAMPLE=VE_22241;TYPE=Complex;DP=445;VD=136;AF=...,445.0,0.3056


,Chr,Start,End,Ref,Alt,AF,DP,Func.refGene,Otherinfo10,Otherinfo11
27128,chr17,41149508,41149509,CT,TG,NaN,NaN,NaN,NaN,NaN


,Chr,Start,End,Ref,Alt,Func.refGene,Otherinfo10,Otherinfo11,DP,AF
24385,chr17,41149508,41149509,CT,TG,exonic,NM5.25,SAMPLE=VE_22241;TYPE=Complex;DP=445;VD=136;AF=...,445.0,0.3056


In [210]:
#TODO: make option to see what is NA and what is not NA
#TODO: make program and functions run on a class
# possible error of pd.to_numeric int out of range may come from long ref/alt strings